## Hello, future Open Crawler user!
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/enterprise-search/elastic-crawler-to-open-crawler-migration.ipynb)

This notebook is designed to help you migrate your Elastic Crawler configurations to Open Crawler-friendly YAML!

We recommend running each cell individually in a sequential fashion, as each cell is dependent on previous cells having been run. Furthermore, we recommend that you only run each cell once as re-running cells may result in errors or incorrect YAML files.

### Setup
First, let's start by making sure `elasticsearch` and other required dependencies are installed and imported by running the following cell:

In [ ]:
!pip install elasticsearch

from getpass import getpass
from elasticsearch import Elasticsearch

import os
import json
import yaml
import pprint

We are going to need a few things from your Elasticsearch deployment before we can migrate your configurations:
- Your **Elasticsearch Endpoint URL**
- Your **Elasticsearch Endpoint Port number**
- An **API key**

You can find your Endpoint URL and port number by visiting your Elasticsearch Overview page in Kibana.

You can create a new API key from the Stack Management -> API keys menu in Kibana. Be sure to copy or write down your key in a safe place, as it will be displayed only once upon creation.

In [ ]:
ELASTIC_ENDPOINT = getpass("Elastic Endpoint: ")
ELASTIC_PORT = getpass("Port")
API_KEY = getpass("Elastic Api Key: ")

es_client = Elasticsearch(
    ":".join([ELASTIC_ENDPOINT, ELASTIC_PORT]),
    api_key=API_KEY,
)

# ping ES to make sure we have positive connection
es_client.info()["tagline"]

Hopefully you received our tagline 'You Know, for Search'. If so, we are connected and ready to go!

If not, please double-check your Cloud ID and API key that you provided above. 

### Step 1: Acquire Basic Configurations

First, we need to establish what Crawlers you have and their basic configuration details.
This migration notebook will attempt to pull configurations for every distinct Crawler you have in your Elasticsearch instance.

In [ ]:
# in-memory data structure that maintains current state of the configs we've pulled
inflight_configuration_data = {}

crawler_configurations = es_client.search(
    index=".ent-search-actastic-crawler2_configurations_v2",
)

crawler_counter = 1
for configuration in crawler_configurations["hits"]["hits"]:
    source = configuration["_source"]

    # extract values
    crawler_oid = source["id"]
    output_index = source["index_name"]

    print(f"{crawler_counter}. {output_index}")
    crawler_counter += 1

    crawl_schedule = (
        []
    )  # either no schedule or a specific schedule - determined in Step 4
    if (
        source["use_connector_schedule"] == False and source["crawl_schedule"]
    ):  # an interval schedule is being used
        print(
            f"   {output_index} uses an interval schedule, which is not supported in Open Crawler!"
        )

    # populate a temporary hashmap
    temp_conf_map = {"output_index": output_index, "schedule": crawl_schedule}
    # pre-populate some necessary fields in preparation for upcoming steps
    temp_conf_map["domains_temp"] = {}
    temp_conf_map["output_sink"] = "elasticsearch"
    temp_conf_map["full_html_extraction_enabled"] = False
    temp_conf_map["elasticsearch"] = {
        "host": "",
        "port": "",
        "api_key": "",
    }
    # populate the in-memory data structure
    inflight_configuration_data[crawler_oid] = temp_conf_map

**Before continuing, please verify in the output above that the correct number of Crawlers was found.**

Now that we have some basic data about your Crawlers, let's use this information to get more configuration values!

### Step 2: URLs, Sitemaps, and Crawl Rules

In the next cell, we will need to query Elasticsearch for information about each Crawler's domain URLs, seed URLs, sitemaps, and crawling rules.

In [ ]:
crawler_ids_to_query = inflight_configuration_data.keys()

crawler_counter = 1
for crawler_oid in crawler_ids_to_query:
    # query ES to get the crawler's domain configurations
    crawler_domains = es_client.search(
        index=".ent-search-actastic-crawler2_domains",
        query={"match": {"configuration_oid": crawler_oid}},
        _source=[
            "name",
            "configuration_oid",
            "id",
            "sitemaps",
            "crawl_rules",
            "seed_urls",
            "auth",
        ],
    )
    print(f"{crawler_counter}.) Crawler ID {crawler_oid}")
    crawler_counter += 1

    # for each domain the Crawler has, grab its config values
    # and update the in-memory data structure
    for domain_info in crawler_domains["hits"]["hits"]:
        source = domain_info["_source"]

        # extract values
        domain_oid = str(source["id"])
        domain_url = source["name"]
        seed_urls = source["seed_urls"]
        sitemap_urls = source["sitemaps"]
        crawl_rules = source["crawl_rules"]

        print(f"    Domain {domain_url} found!")

        # transform seed, sitemap, and crawl rules into arrays
        seed_urls_list = []
        for seed_obj in seed_urls:
            seed_urls_list.append(seed_obj["url"])

        sitemap_urls_list = []
        for sitemap_obj in sitemap_urls:
            sitemap_urls_list.append(sitemap_obj["url"])

        crawl_rules_list = []
        for crawl_rules_obj in crawl_rules:
            crawl_rules_list.append(
                {
                    "policy": crawl_rules_obj["policy"],
                    "type": crawl_rules_obj["rule"],
                    "pattern": crawl_rules_obj["pattern"],
                }
            )

        # populate a temporary hashmap
        temp_domain_conf = {"url": domain_url}
        if seed_urls_list:
            temp_domain_conf["seed_urls"] = seed_urls_list
            print(f"    Seed URls found: {seed_urls_list}")
        if sitemap_urls_list:
            temp_domain_conf["sitemap_urls"] = sitemap_urls_list
            print(f"    Sitemap URLs found: {sitemap_urls_list}")
        if crawl_rules_list:
            temp_domain_conf["crawl_rules"] = crawl_rules_list
            print(f"    Crawl rules found: {crawl_rules_list}")

        # populate the in-memory data structure
        inflight_configuration_data[crawler_oid]["domains_temp"][
            domain_oid
        ] = temp_domain_conf
        print()

### Step 3: Extracting the Extraction Rules

In the next cell, we will find any extraction rules you set for your Elastic Crawlers.

In [ ]:
extraction_rules = es_client.search(
    index=".ent-search-actastic-crawler2_extraction_rules",
    _source=["configuration_oid", "domain_oid", "rules", "url_filters"],
)

extr_count = 1
for exr_rule in extraction_rules["hits"]["hits"]:
    source = exr_rule["_source"]

    config_oid = source["configuration_oid"]
    domain_oid = source["domain_oid"]

    # ensure the config and domain oids actually exist in our in-memory data structure
    if (
        config_oid in inflight_configuration_data
        and domain_oid in inflight_configuration_data[config_oid]["domains_temp"]
    ):

        # initialize extraction rulesets an empty array if it doesn't exist yet
        if (
            not "extraction_rulesets"
            in inflight_configuration_data[config_oid]["domains_temp"][domain_oid]
        ):
            inflight_configuration_data[config_oid]["domains_temp"][domain_oid][
                "extraction_rulesets"
            ] = []

        all_rules = source["rules"]
        all_url_filters = source["url_filters"]

        # extract url filters
        url_filters = []
        if all_url_filters:
            url_filters = [
                {
                    "type": all_url_filters[0]["filter"],
                    "pattern": all_url_filters[0]["pattern"],
                }
            ]

        # extract rulesets
        action_translation_map = {
            "fixed": "set",
            "extracted": "extract",
        }

        ruleset = []
        if all_rules:
            ruleset = [
                {
                    "action": action_translation_map[
                        all_rules[0]["content_from"]["value_type"]
                    ],
                    "field_name": all_rules[0]["field_name"],
                    "selector": all_rules[0]["selector"],
                    "join_as": all_rules[0]["multiple_objects_handling"],
                    "value": all_rules[0]["content_from"]["value"],
                    "source": all_rules[0]["source_type"],
                }
            ]

        temp_extraction_rulesets = {
            "url_filters": url_filters,
            "rules": ruleset,
        }

        print(
            f"{extr_count}.) Crawler {config_oid} has extraction rules {temp_extraction_rulesets}\n"
        )
        extr_count += 1

        inflight_configuration_data[config_oid]["domains_temp"][domain_oid][
            "extraction_rulesets"
        ].append(temp_extraction_rulesets)

### Step 4: Schedules

In the next cell, we will gather any specific time schedules your Crawlers have set. Please note that _interval time schedules_ are not supported by Open Crawler and will be ignored.

In [ ]:
def convert_quartz_to_cron(quartz_schedule: str) -> str:
    _, minutes, hours, day_of_month, month, day_of_week, year = (
        quartz_schedule.split(" ") + [None]
    )[:7]

    # Day of week is 1-7 starting from Sunday in Quartz
    #  and from Monday in regular Cron, adjust:
    # Days before: 1 - SUN, 2 - MON ... 7 - SAT
    # Days after: 1 - MON, 2 - TUE ... 7 - SUN
    if day_of_week.isnumeric():
        day_of_week = (int(day_of_week) - 2) % 7 + 1

    # ignore year
    repackaged_definition = f"{minutes} {hours} {day_of_month} {month} {day_of_week} "

    # ? comes from Quartz Cron, regular cron doesn't handle it well
    repackaged_definition = repackaged_definition.replace("?", "*")
    return repackaged_definition


# ---------------------------------------------------------------

crawler_counter = 1
for crawler_oid, crawler_config in inflight_configuration_data.items():
    output_index = crawler_config["output_index"]

    existing_schedule_value = crawler_config["schedule"]

    if not existing_schedule_value:
        # query ES to get this Crawler's specific time schedule
        schedules_result = es_client.search(
            index=".elastic-connectors-v1",
            query={"match": {"index_name": output_index}},
            _source=["index_name", "scheduling"],
        )
        # update schedule field with cron expression if specific time scheduling is enabled
        if schedules_result["hits"]["hits"][0]["_source"]["scheduling"]["full"][
            "enabled"
        ]:
            quartz_schedule = schedules_result["hits"]["hits"][0]["_source"][
                "scheduling"
            ]["full"]["interval"]
            crawler_config["schedule"] = convert_quartz_to_cron(quartz_schedule)
            print(
                f"{crawler_counter}.) Crawler {output_index} has the schedule {crawler_config['schedule']}"
            )
            crawler_counter += 1

### Step 5: Creating the Open Crawler YAML configuration files

In this final step, we will create the actual YAML files you need to get up and running with Open Crawler!

The next cell performs some final transformations to the in-memory data structure that is keeping track of your configurations.

In [ ]:
# Final transform of the in-memory data structure to a form we can dump to YAML
# for each crawler, collect all of its domain configurations into a list
for crawler_oid, crawler_config in inflight_configuration_data.items():
    all_crawler_domains = []

    for domain_config in crawler_config["domains_temp"].values():
        all_crawler_domains.append(domain_config)
    # create a new key called "domains" that points to a list of domain configs only - no domain_oid values as keys
    crawler_config["domains"] = all_crawler_domains
    # delete the temporary domain key
    del crawler_config["domains_temp"]
    print(f"Transform for {crawler_oid} complete!")

#### **Wait! Before we continue onto creating our YAML files, we're going to need your input on a few things.**

In the next cell, please enter the following details about the _Elasticsearch instance you will be using with Open Crawler_. This instance can be Elastic Cloud Hosted, Serverless, or a local instance.

- The Elasticsearch endpoint URL
- The port number of your Elasticsearch endpoint _(Optional, will default to 443 if left blank)_
- An API key

In [ ]:
ENDPOINT = getpass("Elasticsearch endpoint URL: ")
PORT = getpass("[OPTIONAL] Elasticsearch endpoint port number: ")
OUTPUT_API_KEY = getpass("Elasticsearch API key: ")

# set the above values in each Crawler's configuration
for crawler_config in inflight_configuration_data.values():
    crawler_config["elasticsearch"]["host"] = ENDPOINT
    crawler_config["elasticsearch"]["port"] = int(PORT) if PORT else 443
    crawler_config["elasticsearch"]["api_key"] = OUTPUT_API_KEY

# ping ES to make sure we have positive connection
es_client = Elasticsearch(
    ":".join([ENDPOINT, PORT]),
    api_key=OUTPUT_API_KEY,
)

es_client.info()["tagline"]

#### **This is the final step! You have two options here:**

- The "Write to YAML" cell will create _n_ number of YAML files, one for each Crawler you have.
- The "Print to output" cell will print each Crawler's configuration YAML in the Notebook, so you can copy-paste them into your Open Crawler YAML files manually.

Feel free to run both! You can run Option 2 first to see the output before running Option 1 to save the configs into YAML files.

#### Option 1: Write to YAML file

In [ ]:
# Dump each Crawler's configuration into its own YAML file
for crawler_config in inflight_configuration_data.values():
    base_dir = os.getcwd()
    file_name = (
        f"{crawler_config['output_index']}-config.yml"  # autogen a custom filename
    )
    output_path = os.path.join(base_dir, file_name)

    if os.path.exists(base_dir):
        with open(output_path, "w") as file:
            yaml.safe_dump(crawler_config, file, sort_keys=False)
            print(f" Wrote {file_name} to {output_path}")

#### Option 2: Print to output

In [ ]:
for crawler_config in inflight_configuration_data.values():
    yaml_out = yaml.safe_dump(crawler_config, sort_keys=False)

    print(f"YAML config => {crawler_config['output_index']}-config.yml\n--------")
    print(yaml_out)
    print(
        "--------------------------------------------------------------------------------"
    )

### Next Steps

Now that the YAML files have been generated, you can visit the Open Crawler GitHub repository to learn more about how to deploy Open Crawler: https://github.com/elastic/crawler#quickstart

If you find any problems with this Notebook, please feel free to create an issue in the elasticsearch-labs repository: https://github.com/elastic/elasticsearch-labs/issues